Наименьшее число m, такое, что m! делится без остатка на 10 — это m=5 (5! = 120). Аналогично, наименьшее число m, такое, что m! делится без остатка на 25 — это m=10. В общем случае, значение функции s(n) равно наименьшему числу m, такому что m! без остатка делится на n. Определим функцию S(M, N) = ∑s(n) для всех n ∈ [M, N]. К примеру, S(6, 10) = 3 + 7 + 4 + 6 + 5 = 25. Найдите S(2000000, 2100000).

In [86]:
import math
import functools

In [116]:
def memo(func):
    memo = {}
    @functools.wraps(func)
    def wraper(*args, **kwars):
        key = str(args) + str(kwars)
        if key not in memo:
            memo[key] = func(*args, **kwars)
        return memo[key]
    return wraper

@memo
def fact1(n):
    if n == 0:
        return 1
    else:
        return n * fact(n-1)
    
@memo
def fact(n):
    return math.factorial(n)

@memo
def bin_pow_factorial(n):
    def eratosthenes(N):
        simp = [2]
        nonsimp = set()
        for i in range(3, N + 1, 2):
            if i not in nonsimp:
                nonsimp |= {j for j in range(i * i, N + 1, 2 * i)}
                simp.append(i)
        return simp
    def calc_pow_in_factorial(a, b):
        res = 0
        while a:
            a //= b
            res += a
        return res
    fact_pows = [(x, calc_pow_in_factorial(n, x)) for x in reversed(eratosthenes(n+1))]
    if len(fact_pows) % 2 == 1:
        fact_pows.append((1, 1))
    mul = [fact_pows[i][0] ** fact_pows[i][1] * fact_pows[-i-1][0] ** fact_pows[-i-1][1] for i in range(len(fact_pows)//2)]
    while len(mul) > 1:
        if len(mul) % 2 == 1:
            mul.append(1)
        mul = [mul[i] * mul[-i-1] for i in range(len(mul)//2)]
    return mul[0]

In [117]:
%timeit fact1(500000)


The slowest run took 1430093.47 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 2.37 µs per loop


In [118]:
%timeit fact(500000)

The slowest run took 860917.85 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 3.95 µs per loop


In [119]:
%timeit bin_pow_factorial(500000)

The slowest run took 2008796.53 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 2.37 µs per loop


In [88]:
def s(n):
    w = 1
    while True:
        if bin_pow_factorial(w) % n == 0:
            return w
        w += 1
    

In [89]:
def S(M, N):
    sum = 0
    for i in range(M, N+1):
        sum += s(i)
    return sum

In [90]:
    S(6, 10)

25

In [131]:
def mod(n):
    for i in range(1, n):
        if math.factorial(i) // i == 0:
            return i

In [132]:
%timeit mod(5000)

1 loop, best of 3: 1.98 s per loop


In [135]:
import dis

dis.dis(mod)

  2           0 SETUP_LOOP              52 (to 55)
              3 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               1 (1)
              9 LOAD_FAST                0 (n)
             12 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             15 GET_ITER
        >>   16 FOR_ITER                35 (to 54)
             19 STORE_FAST               1 (i)

  3          22 LOAD_GLOBAL              1 (math)
             25 LOAD_ATTR                2 (factorial)
             28 LOAD_FAST                1 (i)
             31 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             34 LOAD_FAST                1 (i)
             37 BINARY_FLOOR_DIVIDE
             38 LOAD_CONST               2 (0)
             41 COMPARE_OP               2 (==)
             44 POP_JUMP_IF_FALSE       16

  4          47 LOAD_FAST                1 (i)
             50 RETURN_VALUE
             51 JUMP_ABSOLUTE           16
        >>   54 POP_BLOCK
        >>   

In [133]:
def mod1(n):
    for i in range(1, n):
        if math.factorial(i) % i == 0:
            return i

In [134]:
%timeit mod1(5000)

The slowest run took 6.00 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 921 ns per loop


In [136]:
dis.dis(mod1)

  2           0 SETUP_LOOP              52 (to 55)
              3 LOAD_GLOBAL              0 (range)
              6 LOAD_CONST               1 (1)
              9 LOAD_FAST                0 (n)
             12 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             15 GET_ITER
        >>   16 FOR_ITER                35 (to 54)
             19 STORE_FAST               1 (i)

  3          22 LOAD_GLOBAL              1 (math)
             25 LOAD_ATTR                2 (factorial)
             28 LOAD_FAST                1 (i)
             31 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             34 LOAD_FAST                1 (i)
             37 BINARY_MODULO
             38 LOAD_CONST               2 (0)
             41 COMPARE_OP               2 (==)
             44 POP_JUMP_IF_FALSE       16

  4          47 LOAD_FAST                1 (i)
             50 RETURN_VALUE
             51 JUMP_ABSOLUTE           16
        >>   54 POP_BLOCK
        >>   55 LOA

In [142]:
%timeit bin_pow_factorial(2000000)

1 loop, best of 3: 49.9 s per loop


In [141]:
#https://habrahabr.ru/post/255761/#comment_8379739
from timeit import timeit

def naive_factorial(n):
    res = 1
    for i in range(2, n + 1):
        res *= i
    return res

def bin_pow_factorial(n):
    def eratosthenes(N):
        simp = [2]
        nonsimp = set()
        for i in range(3, N + 1, 2):
            if i not in nonsimp:
                nonsimp |= {j for j in range(i * i, N + 1, 2 * i)}
                simp.append(i)
        return simp
    def calc_pow_in_factorial(a, b):
        res = 0
        while a:
            a //= b
            res += a
        return res
    fact_pows = [(x, calc_pow_in_factorial(n, x)) for x in reversed(eratosthenes(n+1))]
    if len(fact_pows) % 2 == 1:
        fact_pows.append((1, 1))
    mul = [fact_pows[i][0] ** fact_pows[i][1] * fact_pows[-i-1][0] ** fact_pows[-i-1][1] for i in range(len(fact_pows)//2)]
    while len(mul) > 1:
        if len(mul) % 2 == 1:
            mul.append(1)
        mul = [mul[i] * mul[-i-1] for i in range(len(mul)//2)]
    return mul[0]



N = 500
REPEATS = 10
setup = 'from __main__ import N, naive_factorial, bin_pow_factorial'
print(timeit(stmt = 'naive_factorial(N)', setup=setup, number=REPEATS)/REPEATS)
print(timeit(stmt = 'bin_pow_factorial(N)', setup=setup, number=REPEATS)/REPEATS)

0.0001017637740005739
0.0003072650175454328


https://habrahabr.ru/post/311908/#comment_9849374
У меня считалось около 0.2 с для диапазона [5300000, 5400000]. Для вашего случая (от 630000000 до 640000000) сейчас проверил, посчиталось примерно за минуту.

Но алгоритм получения числа m придумал несколько иной:

единожды получаем все простые числа в нужном диапазоне (в моём случае от 2 до 5400000), используя решето Эратосфена;
раскладываем число на простые множители (пользуясь полученной таблицей простых чисел) и получаем список с множителями, причём множители могут повторяться;
предполагаем, что число m = 1;
для каждой группы множителей (группой назовём множители имеющие одинаковое значение) считаем количество элементов (N) в группе и выполняем следующее:
берём первое число кратное множителю;
делим его на множитель до тех пор, пока оно делится без остатка;
количество раз, которое его удалось поделить, вычитаем из N;
если N > 0, берем следующее число кратное множителю и переходим в п. 2;
если данное число, кратное множителю, больше m, принимаем его за m и переходим к следующей группе;